In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re
import time
from tqdm.auto import trange
import random
from fake_useragent import UserAgent
import os

In [2]:
# website = 'https://www.car.gr/classifieds/cars/?fs=1&fuel_type=1&fuel_type=2&lang=en&pg=' + str(1)
# response = requests.get(website)
# soup = BeautifulSoup(response.content, 'html.parser')
# results = soup.find_all('li', {'class':''})
# car_page = 'https://www.car.gr' + results[0].find('a', {'class': 'row-anchor'})['href'] + '?lang=en'
# car_response = requests.get(car_page, headers = {'User-Agent':UserAgent().random})
# soup = BeautifulSoup(car_response.content, 'html.parser')

In [3]:
# a = soup.find('div', {'class': 'tw-bg-white tw-rounded tw-shadow-sm tw-my-2 !tw-mt-0', 
#                       'class': 'tw-line-clamp-2 tw-text-ellipsis classified-title-container'})
# a.find('h1', {'class': 'tw-text-2xl tw-mb-0 classified-title tw-mt-2 tw-block'}).get_text().split('    ')

- ### Extract whole page

In [4]:
car_dict = dict({'title': [],
                 'price': [],
                 'registration': [],
                 'km': [],
                 'displacement': [],
                 'hp': [],
                 'fuel': [],
                 'transmission': [],
                 'category': [],
                 'location': [],
                 'link': []})

In [ ]:
for i in trange(120, desc = 'Loop over pages'):
    
    website = 'https://www.car.gr/classifieds/cars/?fs=1&fuel_type=1&fuel_type=2&lang=en&pg=' + str(i)
    response = requests.get(website)
    try:
        assert response.status_code == 200, print(f'Listings Page: {response.status_code}')
    except:
        time.sleep(random.uniform(20, 40))
        continue
    soup = BeautifulSoup(response.content, 'html.parser')
    results = soup.find_all('li', {'class':''})

    for j in trange(len(results), desc = 'Loop over listings', leave=False):
        
        # listing title
        title = results[j].find('h2', {'class': 'title'}).get_text()
        title = title.split('\n')[1]
        # price
        try:
            price = results[j].find('div', {'class': 'price-tag current-price'}).get_text()
            price = ''.join(re.split(r'\W', price)[1:3])
        except:
            # Ask price case
            price = 'Ask price'
        # location
        info = results[j].find_all('span', {'class': ''})[3]
        location = info.get_text()
        

        car_page = 'https://www.car.gr' + results[j].find('a', {'class': 'row-anchor'})['href'] + '?lang=en'
        car_response = requests.get(car_page,
                                    headers = {'User-Agent':UserAgent().random})
        time.sleep(5)
        try:
            assert car_response.status_code == 200, print(f'Car Page: {car_response.status_code}')
        except:
            time.sleep(random.uniform(25, 45))
            if car_response.status_code == 429:
                break
            else:
                continue

        soup = BeautifulSoup(car_response.content, 'html.parser')
        car_results = soup.find_all('div', {'class': 'tw-p-4', 
                                            'class': 'tw-relative tw-w-full tw-flex tw-col-span-3 lg:tw-col-span-3 sm:tw-col-span-4 md:tw-col-span-4'})
        if len(car_results) != 0:
            registration = car_results[0].get_text().split('        ')[5]
            km = car_results[1].get_text().split('        ')[5]
            fuel = car_results[2].get_text().split('        ')[5]
            if 'Petrol' in fuel or 'Diesel' in fuel:
                try:
                    displacement = car_results[3].get_text().split('        ')[5]
                    hp = car_results[4].get_text().split('        ')[5]
                    transmission = car_results[6].get_text().split('        ')[5]
                    category = car_results[7].get_text().split('        ')[5]

                    car_dict['title'].append(title)
                    car_dict['price'].append(price)
                    car_dict['registration'].append(registration)
                    car_dict['km'].append(km)
                    car_dict['displacement'].append(displacement)
                    car_dict['hp'].append(hp)
                    car_dict['fuel'].append(fuel)
                    car_dict['transmission'].append(transmission)
                    car_dict['category'].append(category)
                    car_dict['location'].append(location)
                    car_dict['link'].append(car_page)
                except:
                    continue
        # Waiting time before changing car page 
        time.sleep(random.uniform(7, 14))
    # Waiting time before changing listings page 
    time.sleep(random.uniform(50, 70))

Loop over pages:   0%|          | 0/120 [00:00<?, ?it/s]

Loop over listings:   0%|          | 0/26 [00:00<?, ?it/s]

Car Page: 403


Loop over listings:   0%|          | 0/26 [00:00<?, ?it/s]

Car Page: 403


Loop over listings:   0%|          | 0/26 [00:00<?, ?it/s]

In [ ]:
df = pd.DataFrame(car_dict).iloc[1:, :]
for i in range(1,100):
    file = f'listings_{i}.csv'
    if not os.path.isfile(file):
        df.to_csv(file)
        break

In [ ]:
df.shape

In [ ]:
dfs = []
for i in range(1, 100):
    file = f'listings_{i}.csv'
    try:
        df = pd.read_csv(file, index_col = 0)
        dfs.append(df)
    except:
        break

df  = pd.concat(dfs, axis = 0)

In [ ]:
df.shape

In [ ]:
df.drop_duplicates(keep = 'last', inplace = True)
df.shape

In [ ]:
df['registration'] = [x[:-1] for x in df['registration'].values]
df['km'] = [x[:-1] for x in df['km'].values]
df['hp'] = [x[:-1] for x in df['hp'].values]
df['fuel'] = [x[:-1] for x in df['fuel'].values]
df['transmission'] = [x[:-1] for x in df['transmission'].values]
df['category'] = [x[:-1] for x in df['category'].values]

In [ ]:
df.tail()

In [ ]:
df['category'].unique()

In [ ]:
df.groupby(['category']).size()

In [ ]:
import matplotlib.pyplot as plt

plt.bar(df.groupby(['category']).size().index, df.groupby(['category']).size().values)
for i in range(len(df.groupby(['category']).size().index)):
    plt.text(i, df.groupby(['category']).size().values[i] + 10, df.groupby(['category']).size().values[i], ha = 'center')
plt.xlabel('Category')
plt.ylabel('Occurencies')
plt.xticks(rotation = 90)
plt.figure(figsize=(10,15))
plt.tight_layout();

In [ ]:
# try:
#     registration = car_results[2].get_text().split(' ')[-1]
# except:
#     registration = None
# try:
#     km = car_results[3].get_text().split(' ')[2]
# except:
#     km = None
# try:
#     displacement = car_results[5].get_text().split(' ')[4]
# except:
#     displacement = None
# try:
#     hp = car_results[5].get_text().split(' ')[7]
# except:
#     hp = None
# try:
#     fuel = car_results[4].get_text().split(' ')[-1]
# except:
#     fuel = None
# try:
#     transmission = car_results[6].get_text().split(' ')[-1]
# except:
#     transmission = None
# try:
#     category = car_results[7].get_text().split(' ')[-1]
# except:
#     category = None

# info = results[j].find('div', {'class':'d-flex justify-content-between'}).get_text().split('\n  ')

# try:
#     location = info[2].split('  ')
#     location = location[0].split(' ')[1:]
#     location = ' '.join(location)
# except:
#     location = None